# Get started

In this tutorial, we will go through how to prepare for the inputs required by TraSig, using user defined trajectory. Regardless of the pseudotime trajectory tool you used, to prepare the inputs for TraSig, you need:

    * Expression matrix (row: cells; column: genes)
    * Cluster / branch / edge label assigned to each cell
    * Progression on the branch assigned to each cell, a value in [0, 1]
    * Cell type label assigned to each cell (for visualization purpose only)
    * Sampling time of cells (optional; if unknown, then all set to 0)

To prepare for the expression matrix starting from the counts, after filtering no-quality cells / genes. You may normalize the counts using TPM (we use a scaling factor of 1e4) and then transform the values using log (x + 1).

After obtaining these, follow the steps below to prepare the inputs for TraSig. We will use ti_slingshot (Slingshot in dynverse) on the dataset "oligodendrocyte-differentiation-clusters_marques.rds" as an example. 

Note that the emphasis of this tutorial is to **illustarte the input file formats**. You may use your own prefered pseudotime trajectory tools (not necessarily those implemented in dynverse). If you would like to learn the details on how to prepare inputs from dynverse trajectory outputs, refer to [Prepare_input_from_dynverse_ti_methods](Prepare_input_from_dynverse_ti_methods.ipynb) in the tutorials. 

**Table of Content**
1. [Load expression and trajectory results](#1)
2. [Prepare pseudotime results per cell for TraSig](#2)
3. [Subsetting expression data (to keep only ligand-receptors)](#3)
4. [Save correspondence from sampling time to paths](#4)

**Extra Package Requirements**
* h5py >= 3.1.0 (required to load dynverse trajectory results)
* rpy2 >= 3.3.6 (required to load dynverse datasets)
* matplotlib-base >= 3.3.4 (required for plotting)
* scikit-learn >= 0.23.2 (required for evaluating trajectory results)
* scipy >= 1.5.4 (required to prepare sampling time input)


In [1]:
import os, sys
import argparse
import time
from os.path import exists
import collections
from typing import Iterable
import pickle
from collections import Counter
import requests

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import h5py 
import rpy2.robjects as robjects  

In [2]:
# example data set 
project = "oligodendrocyte-differentiation-clusters_marques"

# set the path to the inputs for the trajectory inference (e.g. expression)
input_path = "../trajectory/input"

# set the path to save the outputs of this script (place to save inputs for TraSig)
output_path = "../example/input"

# set the path to the trajectory output
trajectory_filename = f"../trajectory/output/output.h5"

In [3]:
# set the names for output files 
preprocess = "None"
model_name = "ti_slingshot"
others = "None"

if preprocess != "None":
    _preprocess = f"_{preprocess}"
else:
    _preprocess = ""
    
if others == "None":
    condition = ""
        
suffix = f"{_preprocess}_{model_name}{condition}"

# 1. Load expression and trajectory results

* You may need to customize your code in this part to load the expression matrix and the trajectory results from the specific pseudotime inference tool you used. 

<a id=1></a>

In [4]:
filepath = f"{input_path}/{project}.rds"

if os.path.exists(filepath):
    pass 
else:
    url = f"https://zenodo.org/record/1443566/files/real/silver/{project}.rds?download=1"
    r = requests.get(url)  

    with open(filepath, 'wb') as f:
        f.write(r.content)
        
filepath = f"{input_path}/{project}.rds"

from rpy2.robjects import pandas2ri
pandas2ri.activate()

readRDS = robjects.r['readRDS']
df = readRDS(filepath)
# df = pandas2ri.rpy2py_dataframe(df)
data_keys = list(df.names)

cell_ids = df[data_keys.index('cell_ids')]
expression = df[data_keys.index('expression')]  
genes = df[data_keys.index('feature_info')]['feature_id'].values

N = len(cell_ids)  # number of cells 
G = len(genes)  # number of genes 

# load true traejctory and labels 
# true trajectory
milestones_true = df[data_keys.index('milestone_ids')]
network_true = df[data_keys.index('milestone_network')]
M_true = len(milestones_true)

# add node index; node index consistent with index in 'milestone_ids'
# will use node index to present node from now on
network_true['idx_from'] = [list(milestones_true).index(i) for i in network_true['from']]
network_true['idx_to'] = [list(milestones_true).index(i) for i in network_true['to']]

membership_true = df[data_keys.index('milestone_percentages')] 
# assign cells to the most probable node 
assignment_true = membership_true[membership_true.groupby(['cell_id'])['percentage'].transform(max) == membership_true['percentage']]
assignment_true.set_index('cell_id', inplace=True)
assignment_true = assignment_true.reindex(cell_ids)
clusters_true = [list(milestones_true).index(c) for c in assignment_true['milestone_id'].values]

# load trajectory inference results 
f = h5py.File(trajectory_filename, 'r')

# # Check what keys are 
# for key in f.keys():
#     print(key) 

key = 'data'

# Get the HDF5 group
group = f[key]

# #Checkout what keys are inside that group.
# for key in group.keys():
#     print(key)

_percentages = group['milestone_percentages']
_network = group['milestone_network']
_progressions = group['progressions']

# # Check what keys are  
#  data.keys()
#  data['data'].keys()

_cell_ids = list(_percentages['data']['cell_id'])
_cell_ids = [i.decode('utf-8') for i in _cell_ids]
estimated_percentages = pd.DataFrame(zip(_cell_ids, list(_percentages['data']['milestone_id']), list(_percentages['data']['percentage'])))
estimated_percentages.columns = ['cell_id', 'milestone_id', 'percentage']

_cell_ids = list(_progressions['data']['cell_id'])
_cell_ids = [i.decode('utf-8') for i in _cell_ids]
estimated_progressions = pd.DataFrame(zip(_cell_ids, list(_progressions['data']['from']), list(_progressions['data']['to']), list(_progressions['data']['percentage'])))
estimated_progressions.columns = ['cell_id', 'from', 'to' ,'percentage']
estimated_progressions = estimated_progressions.set_index("cell_id")  
estimated_progressions = estimated_progressions.reindex(assignment_true.index.values)  # assignment_true already reindexed by cell_ids

estimated_network = pd.DataFrame(pd.DataFrame(zip(list(_network['data']['from']), list(_network['data']['to']), list(_network['data']['length']))))
estimated_clusters = estimated_percentages.loc[estimated_percentages.groupby(["cell_id"])["percentage"].idxmax()].set_index('cell_id').reindex(cell_ids)
estimated_clusters['milestone_id'] = [_c.decode("utf-8") for _c in estimated_clusters['milestone_id']]

# 2. Prepare pseudotime results per cell for TraSig
<a id=2></a>

In this part, we prepare the following for TraSig:

    1. assigned path (edge)
    2. assigned time / progression on the edge 
    3. cell type labels (ground truth)

* You may need to customize your code in this part to load the corresponding trajectory results from the specific pseudotime inference tool you used. 

In [5]:
estimated_progressions['from'] = [i.decode('utf-8') for i in estimated_progressions['from']] 
estimated_progressions['to'] = [i.decode('utf-8') for i in estimated_progressions['to']] 
estimated_progressions['edge'] = estimated_progressions['from'] + '_' + estimated_progressions['to'] 

# assign unique label (integer) to each edge 

edges = np.unique(estimated_progressions['edge'])

edge2idx = {}
for i, v in enumerate(edges):
    edge2idx[v] = i
    
estimated_progressions['idx_edge'] = estimated_progressions['edge'].replace(edge2idx)

hid_var = {'cell_path': estimated_progressions['idx_edge'].values,
          'cell_time': estimated_progressions['percentage'].values,
          'cell_labels':assignment_true['milestone_id'].values} 

### Input file format
This is dictionary with three keys, each correponding to a numpy array. 

    * "cell_path" (integer or strings): cluster / branch / edge label of each cell 
    * "cell_time" (float): progression on the branch assigned to each cell, a value in [0, 1]
    * "cell_labels" (integer or strings): cell type label assigned to each cell 
 
See following for an example.

In [7]:
hid_var

{'cell_path': array([0, 1, 1, ..., 1, 1, 1]),
 'cell_time': array([0.        , 0.        , 0.        , ..., 0.87017241, 0.        ,
        0.97737995]),
 'cell_labels': array(['OPC', 'OPC', 'OPC', ..., 'NFOL', 'OPC', 'NFOL'], dtype=object)}

### Filename format

Remember to save this dictionary using the following file format.

In [8]:
# save 
filename = f"{project}{_preprocess}_{model_name}_it2_hid_var.pickle"
with open(os.path.join(output_path, filename), 'wb') as handle:
    pickle.dump(hid_var, handle, protocol=pickle.HIGHEST_PROTOCOL)

## 3. Subsetting expression data (to keep only ligand-receptors )
<a id=3></a>

1. the following take expression and ligand-receptor list (database) as input
2. make sure your expression matrix is a numpy array (row: cells; column: genes)

In [24]:
# get interaction file (list of (ligand, receptor))
lr_list_path = "../ligand_receptor_lists"
list_type = 'ligand_receptor'
filename = f"{list_type }_FANTOM.pickle"

with open(os.path.join(lr_list_path, filename), 'rb') as handle:
    interaction_list = pickle.load(handle)
    
ligands_receptors = np.unique([i[0] for i in interaction_list] + [i[1] for i in interaction_list])

# get list of genes identified as ligand or receptor 
genes_upper = [g.upper() for g in genes]
kepted_genes = list(set(genes_upper).intersection(set(ligands_receptors)))

df = pd.DataFrame(expression)
df.columns = genes_upper
df.index = cell_ids

df_sub = df[kepted_genes]

# save filtered expression 
filename = f"{project}{_preprocess}_{list_type}.txt"
data_file = os.path.join(output_path, filename)
df_sub.to_csv(data_file)


# save filtered interactions (list of (ligand, receptor) that are expressed)
filtered_interactions = []
for i, j in interaction_list:
    if i in kepted_genes and j in kepted_genes:
        filtered_interactions.append((i, j))
 

filename = f"{list_type}_{project}{_preprocess}.pickle"
with open(os.path.join(output_path, filename), 'wb') as handle:
    pickle.dump(filtered_interactions, handle, protocol=pickle.HIGHEST_PROTOCOL)


## 4. Save correspondence from sampling time to paths
<a id=4></a>

1. Note here cell_path refers to the edge where the cell is assigned to 
2. We will only find interactions between cells from the same sampling time and those from consecutive sampling times:
    - i.e., between the ones from the same time, the ones from 1 sampling time before the ones from 1 sampling time after
3. Given we don't know the sampling time for the example data, we set all sampling time as 0. For your own data, if you are not certain about sampling time, just assign the time for all cells as 0.
4. If sampling time is known, rank the real time (e.g. day 0, day 17) first and assign the rank to the cell_ori_time variable below. 
    - e.g., for cells from two sampling time day 0 and day 17, assign those from day 0 as 0 and those from day 17 as 1. 

In [26]:
from scipy import stats

#### If known sampling time, then set the following variable = the sampling time of cells

In [27]:
cell_ori_time = np.repeat(0, N)  # put all cells at time 0 if sampling time unknow 

#### The following is trying to assign each cluster / branch / edge a sampling time, determined by the majority of cells

In [28]:
unique_days = np.unique(cell_ori_time)
sorted_days = list(np.sort(unique_days)) 
cell_paths = np.unique(hid_var["cell_path"])

In [29]:
sampleT2path = dict.fromkeys(range(len(sorted_days)))  # use index of sorted sampling time as key
for k, v in sampleT2path.items():
    sampleT2path[k] = []

In [30]:
for i, cur_path in enumerate(cell_paths):
    print("current path (edge)", cur_path)
    
    # get data corresponding to a path
    condition = hid_var["cell_path"] == cur_path
    cur_days = np.array(cell_ori_time)[condition]
    
    # get the sampling time for the majority cells 
    mode, count = stats.mode(cur_days)
    print(f"Sampling time for the majority of cells: {mode[0]}, making {round(float(count[0])/len(cur_days), 2)}% percent")
    cur_sampleT = mode[0]
    
    # will use index instead of input time 
    sampleT2path[sorted_days.index(cur_sampleT)].append(cur_path)
    

current path (edge) 0
Sampling time for the majority of cells: 0, making 1.0% percent
current path (edge) 1
Sampling time for the majority of cells: 0, making 1.0% percent
current path (edge) 2
Sampling time for the majority of cells: 0, making 1.0% percent
current path (edge) 3
Sampling time for the majority of cells: 0, making 1.0% percent


### Input file format
This is dictionary with sampling time as keyes, each correponding to a list of clusters that are deterimined to be sampled from the sampling time. 
 
See following for an example.

In [31]:
sampleT2path

{0: [0, 1, 2, 3]}

In [28]:
# save the dictionary
filename = 'sampling_time_per_path_' + project + suffix + '.pickle'

with open(os.path.join(output_path, filename), 'wb') as handle:
    pickle.dump(sampleT2path, handle, protocol=pickle.HIGHEST_PROTOCOL)